## Morphology locations as a Form

This is probably not very useful. Morphology locations are a rather abstract concept.
The form simply plots the generate locations on the morphology and writes them to a csv file.

In [ ]:
import obi_one as obi
from pathlib import Path

morph_path = Path("../data/morphologies/L5_TPC:A_MusMusculus_PrimarySomatosensory_C060114A5/C060114A5.swc")

In [ ]:
morph_locations = obi.RandomMorphologyLocations(number_of_locations=[10, 20], section_types=(4, ))
# Try this one for a more exciting example.
# morph_locations = obi.ClusteredPathDistanceMorphologyLocations(
#                         number_of_locations=[100, 200],
#                         section_types=(4, ),
#                         n_clusters=10,
#                         cluster_max_distance=20,
#                         path_dist_mean=200,
#                         path_dist_sd=80,
#                         n_groups_per_cluster=2,
#                     )
morph_loc_scan_config = obi.MorphologyLocationsScanConfig(
                    initialize=obi.MorphologyLocationsScanConfig.Initialize(
                        morphology=[morph_path]
                    ),
                    morph_locations=morph_locations
)

grid_scan = obi.GridScanGenerationTask(form=morph_loc_scan_config, 
                         output_root="../../../obi-output/morphology_locations")
grid_scan.execute()
obi.run_tasks_for_generated_scan(grid_scan)

## Morphology locations blocks

There are several blocks of differing complexity that generate locations. Above, we used the simplest one, but others are available.

Here, we demo most of them.

In [ ]:
from morphio import Morphology as MorphioMorphology
m = MorphioMorphology(morph_path)

### RandomMorphologyLocations
Simplest case: Random locations without constraints.

Note that there are no constraints on path distance or anything, really.

In [ ]:
# Default number of locations is 1
tst = obi.RandomMorphologyLocations()
display(tst.points_on(m))

# Can specify a larger number instead.
tst = obi.RandomMorphologyLocations(number_of_locations=10)
display(tst.points_on(m))

# Can also limit the section types.
# Note that the above likely has different section types, this one only shows "4".
tst = obi.RandomMorphologyLocations(number_of_locations=10, section_types=(4, ))
display(tst.points_on(m))

### RandomGroupedMorphologyLocations
This only adds a grouping to the output. That is, the column "source_index" can have values other than 0.

In [ ]:
# Default is a single group. This does not change anything.
tst = obi.RandomGroupedMorphologyLocations()
display(tst.points_on(m))

# More customized. Now "source_index" has different values
tst = obi.RandomGroupedMorphologyLocations(number_of_locations=10, n_groups=2)
display(tst.points_on(m))

### ClusteredMorphologyLocations
This constrains generated locations such that they are clustered. That means, groups of them will be within a specified maximum distance of each other. Note that this distance is two times the value of the parameter 'cluster_max_distance'.

In [ ]:
# Note that many locations will be on the same section and nearby segments.
tst = obi.ClusteredMorphologyLocations(number_of_locations=20, n_clusters=5, cluster_max_distance=5.0)
display(tst.points_on(m))

# Also section type can always be specified.
tst = obi.ClusteredMorphologyLocations(number_of_locations=20, n_clusters=5,
                                       cluster_max_distance=5.0, section_types=(3,))
display(tst.points_on(m))

### ClusteredPathDistanceMorphologyLocations
This exposes the full complexity: Several clusters at specific path distances, and several groups per cluster.

In [ ]:
tst = obi.ClusteredPathDistanceMorphologyLocations(
    number_of_locations=100,
    n_clusters=5,
    cluster_max_distance=10.0,
    path_dist_mean=100.0,
    path_dist_sd=2.0
)
display(tst.points_on(m))
